In [45]:
# !pip install azure-storage-blob
# !pip install python-dotenv

import os
from dotenv import load_dotenv
from setup_utils import fetch_data, load_data, create_time_index
from datetime import datetime
import pandas as pd
import numpy as np

CONNECTION_STRING = os.getenv("CONNECTION_STRING")

load_dotenv()
fetch_data(CONNECTION_STRING)

(
    brand_mapping_backup,
    macro_data_backup,
    brand_constraint_backup,
    pack_constraint_backup,
    segment_constraint_backup,
    sales_data_backup,
    volume_variation_constraint_backup,
) = load_data()

(
    macro_data_backup,
    sales_data_backup,
) = create_time_index([macro_data_backup, sales_data_backup])

from sklearn.metrics import make_scorer, r2_score

pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)

data/brand_segment_mapping_hackathon.xlsx
data/macro_data.xlsx
data/maximum_discount_constraint_hackathon.xlsx
data/sales_data_hackathon.xlsx
data/volume_variation_constraint_hackathon.xlsx
data/submission_template_hackathon.csv


In [46]:
import tensorflow as tf

In [47]:
brand_mapping = brand_mapping_backup.copy(deep=True)
macro_data = macro_data_backup.copy(deep=True)
brand_constraint = brand_constraint_backup.copy(deep=True)
pack_constraint = pack_constraint_backup.copy(deep=True)
segment_constraint = segment_constraint_backup.copy(deep=True)
sales_data = sales_data_backup.copy(deep=True)
volume_variation_constraint = volume_variation_constraint_backup.copy(deep=True)

sales_index = sales_data.index.unique()
macro_data = macro_data.loc[sales_index].sort_index()
covid = pd.Series([1 if (i<=datetime(2020,5,1) and i>=datetime(2020,3,1)) else 0 for i in macro_data.index], index=sales_index, name="covid")
macro_data = macro_data.join(covid)

In [48]:
macro_data = macro_data.loc[sales_index].sort_index()
macro_data = (macro_data/macro_data.mean()-1).copy(deep=True)
macro_data = macro_data.astype(np.float64).values
macro_data = np.expand_dims(macro_data, 1)

In [49]:
nr_data = (
    sales_data
    .reset_index()
    .groupby(["date", "sku"])
    .net_revenue.sum()
    .sort_index()
    .unstack(1)
    .clip(0.0, None)
    .fillna(0.0)
    .astype(np.float64)
    .values
)
nr_data_mask = (
    sales_data
    .reset_index()
    .groupby(["date", "sku"])
    .net_revenue.sum()
    .sort_index()
    .unstack(1)
    .applymap(lambda x: x if x>=0 else np.nan)
    .notna()
    .astype(np.float64)
    .values
)

nr_shifted = (
    sales_data
    .reset_index()
    .groupby(["date", "sku"])
    .net_revenue.sum()
    .sort_index()
    .unstack(1)
    .applymap(lambda x: x if x>=0 else np.nan)
    .clip(0.0, None)
    .shift(1)
    .fillna(method="bfill")
    .fillna(0.0)
    .astype(np.float64)
    .values
)

volume_data = (
    sales_data
    .reset_index()
    .groupby(["date", "sku"])
    .volume.sum()
    .sort_index()
    .unstack(1)
    .clip(0.0, None)
    .fillna(0.0)
    .astype(np.float64)
    .values
)


discount_data = (
    sales_data
    .reset_index()
    .groupby(["date", "sku"])[["promotional_discount", "other_discounts"]].sum()
    .sort_index()
    .stack()
    .unstack(1)
    .fillna(0.0)
    .clip(None, 0)
)
discount_data = -np.swapaxes(discount_data.astype(np.float64).values.reshape(55,2,discount_data.shape[1]), 1, 2)

In [50]:
scaler = nr_data.mean()
vol_scaler = volume_data.mean()

nr_data = nr_data/scaler
discount_data = discount_data/scaler
nr_shifted = nr_shifted/scaler

volume_data = volume_data/vol_scaler

In [51]:
time_index_array = np.expand_dims(np.arange(1, macro_data.shape[0]+1), 1)/100

In [90]:
tf.compat.v1.reset_default_graph()
tf.compat.v1.enable_eager_execution()

y = tf.constant(nr_data, dtype=tf.float64)
y_mask = tf.constant(nr_data_mask, dtype=tf.float64)

discounts = tf.constant(discount_data, dtype=tf.float64)
mixed_effect = tf.constant(macro_data, dtype=tf.float64)
time_index = tf.constant(np.expand_dims(np.arange(1, macro_data.shape[0]+1), 1), dtype=tf.float64)
shifted_nr = tf.constant(nr_shifted, dtype=tf.float64)
y_vol = tf.constant(volume_data, dtype=tf.float64)

val_splitter_ = tf.constant(5, dtype=tf.int32)
val_splitter = 5 #if val_splitter_ == 5 else 2

In [91]:
tf.compat.v1.reset_default_graph()
tf.compat.v1.disable_eager_execution()

sess = tf.compat.v1.Session()

#Y
y = tf.compat.v1.placeholder(dtype=tf.float64, name="nr_actual")
y_mask = tf.compat.v1.placeholder(dtype=tf.float64, name="nr_mask")

# X
discounts = tf.compat.v1.placeholder(dtype=tf.float64, name="discounts")
mixed_effect = tf.compat.v1.placeholder(dtype=tf.float64, name="mixed_effects")
time_index = tf.compat.v1.placeholder(dtype=tf.float64, name="time_index")
# shifted_nr = tf.compat.v1.placeholder(dtype=tf.float64, name="shifted_nr")
y_vol = tf.compat.v1.placeholder(dtype=tf.float64, name="volume_actual")

val_splitter_ = tf.compat.v1.placeholder(dtype=tf.int32)
val_splitter = 5 #if val_splitter_ == 5 else 2

In [92]:
dim_size = (1,nr_data.shape[1])
me_size = macro_data.shape[-1]

baseline_intercept = tf.Variable(np.expand_dims((nr_data.mean(0)*0.3), 0), dtype=tf.float64)

baseline_slope1_global = tf.Variable(np.full((1,1), 0.1), dtype=tf.float64)
baseline_slope1_hier = tf.Variable(np.full(dim_size, 0.1), dtype=tf.float64)
baseline_slope1 = baseline_slope1_global + baseline_slope1_hier

# baseline_slope2_global = tf.Variable(np.full((1,1), 0.1), dtype=tf.float64)
# baseline_slope2_hier = tf.Variable(np.full(dim_size, 0.1), dtype=tf.float64)
# baseline_slope2 = baseline_slope2_global + baseline_slope2_hier

mixed_effect_mult_global = tf.Variable(np.random.normal(loc=0, size=(1, 1, me_size)), dtype=tf.float64)
mixed_effect_mult_hier = tf.Variable(np.random.normal(loc=0, size=(*dim_size, me_size)), dtype=tf.float64)
mixed_effect_mult = mixed_effect_mult_global + mixed_effect_mult_hier

discount_slope_global = tf.math.sigmoid(tf.Variable(np.random.normal(loc=0, size=(1, 1, 2)), dtype=tf.float64))*3
discount_slope_hier = tf.math.sigmoid(tf.Variable(np.random.normal(loc=0, size=(*dim_size, 2)), dtype=tf.float64))*3
discount_slope = discount_slope_global + discount_slope_hier

roi_mults_global = tf.Variable(np.random.normal(loc=0, size=(1, 1, me_size)), dtype=tf.float64)
roi_mults_hier = tf.Variable(np.random.normal(loc=0, size=(*dim_size, me_size)), dtype=tf.float64)
roi_mults = roi_mults_global + roi_mults_hier

nr_to_vol_slope = tf.Variable(np.random.normal(loc=0, size=dim_size), dtype=tf.float64)


hier_var_list = [baseline_slope1_hier, mixed_effect_mult_hier, discount_slope_hier, roi_mults_hier] #baseline_slope2_hier
global_var_list = [baseline_slope1_global, mixed_effect_mult_global, discount_slope_global, roi_mults_global, nr_to_vol_slope] #baseline_slope2_global

In [93]:
# impacts
base1 = tf.multiply(baseline_slope1, time_index) + baseline_intercept
base2 = base1 #+ tf.multiply(baseline_slope2, shifted_nr)
mixed_effect_impact = 1 + tf.nn.tanh(tf.multiply(mixed_effect, mixed_effect_mult))
total_mixed_effect_impact = tf.reduce_prod(mixed_effect_impact, axis=-1)
discount_impact = tf.multiply(discount_slope, discounts)
roi_mult_impact = 1 + tf.nn.tanh(tf.multiply(mixed_effect_impact, roi_mults))
total_roi_mult_impact = tf.expand_dims(tf.reduce_prod(roi_mult_impact, axis=1), axis=1)


In [94]:
@tf.function
def wape(y_actual, y_prediction):
    return tf.reduce_sum(tf.math.abs(y_actual - y_prediction))/tf.reduce_sum(y_actual)

@tf.function
def mse(y_actual, y_prediction):
    return tf.reduce_sum(tf.math.square(y_actual - y_prediction))

In [95]:
# prediction
y_pred = tf.multiply(
    y_mask,
    (
        tf.multiply(base2, total_mixed_effect_impact)
        + tf.reduce_sum(discount_impact, axis=-1)
    )
)

y_vol_pred = tf.multiply(y_pred, nr_to_vol_slope)

y_split = tf.split(y, val_splitter)
y_pred_split = tf.split(y_pred, val_splitter)

y_vol_split = tf.split(y_vol, val_splitter)
y_vol_pred_split = tf.split(y_vol_pred, val_splitter)


# loss
total_wape = tf.math.reduce_mean([wape(y_split[i], y_pred_split[i]) for i in range(0,val_splitter)])
total_mse = mse(y, y_pred)
actual_wape = wape(y, y_pred)

total_wape_vol = tf.math.reduce_mean([wape(y_vol_split[i], y_vol_pred_split[i]) for i in range(0,val_splitter)])
total_mse_vol = mse(y_vol, y_vol_pred)
actual_wape_vol = wape(y_vol, y_vol_pred)


reg1 = sum([tf.reduce_sum(tf.square(i)) for i in hier_var_list])
reg2 = sum([tf.reduce_sum(tf.square(i)) for i in global_var_list])

loss = (
    1e3*total_wape_vol
    +1e1*total_mse_vol
    +1e3*total_wape
    +1e1*total_mse
    +1e3*reg2
    +1e1*reg1
)

In [96]:
splitter = 40

feed_dict1 = {
    discounts : discount_data[:splitter],
    mixed_effect: macro_data[:splitter],
    y_vol : volume_data[:splitter],
    y : nr_data[:splitter],
    # shifted_nr : nr_shifted[:splitter],
    y_mask : nr_data_mask[:splitter],
    time_index : time_index_array[:splitter],
    val_splitter_ : 5
}

feed_dict2 = {
    discounts : discount_data[splitter:-5],
    mixed_effect: macro_data[splitter:-5],
    y_vol : volume_data[splitter:-5],
    y : nr_data[splitter:-5],
    # shifted_nr : nr_shifted[splitter:-5],
    y_mask : nr_data_mask[splitter:-5],
    time_index : time_index_array[splitter:-5],
    val_splitter_ : 5
}

feed_dict3 = {
    discounts : discount_data[-2:],
    mixed_effect: macro_data[-2:],
    y_vol : volume_data[-2:],
    y : nr_data[-2:],
    # shifted_nr : nr_shifted[-2:],
    y_mask : nr_data_mask[-2:],
    time_index : time_index_array[-2:],
    val_splitter_ : 5
}


In [97]:
epoch = 0
# optimizer
lr = lambda x : 1 / np.power(x/5 + 10, 1/2)
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=lr(epoch))#, beta1=0.1, beta2=0.1)
train = optimizer.minimize(loss)

# initialize variables
init = tf.compat.v1.global_variables_initializer()
sess.run(init, feed_dict1)

In [98]:
[lr(i) for i in [0, 1, 10, 100, 1000, 10000, 20000]]#, 50000, 80000]]

[0.31622776601683794,
 0.31311214554257477,
 0.2886751345948129,
 0.18257418583505536,
 0.06900655593423542,
 0.022304986837273527,
 0.015791661046371634]

In [99]:
metric_update_track = {
    "epoch" : [],
    "actual_wape" : [],
    "test_wape" : [],
    "loss" : [],
    "mse" : [],
    "reg1" : [],
    "reg2" : []
}

# train model
num_epochs = 20000
for epoch in range(num_epochs):
    (
        _,
        current_loss,
        current_wape,
        # current_mse,
        current_wape_vol,
        # current_mse_vol,
        current_reg1,
        current_reg2
    )= sess.run([
        train,
        loss,
        actual_wape,
        # total_mse,
        actual_wape_vol,
        # total_mse_vol,
        reg1,
        reg2
    ], feed_dict1)

    current_wape_test, current_wape_vol_test = sess.run([actual_wape, actual_wape_vol], feed_dict2)


    if (epoch + 1) % 250 == 0:
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {current_loss:.4f}, WAPE: {current_wape:.4f}, WAPE_TEST: {current_wape_test:.4f}, WAPE_VOL: {current_wape_vol:.4f}, WAPE_VOL_TEST: {current_wape_vol_test:.4f}, reg1: {current_reg1:.4f}, reg2: {current_reg2:.4f}")
        # metric_update_track["epoch"].append(epoch)
        # metric_update_track["actual_wape"].append(current_wape)
        # metric_update_track["test_wape"].append(current_wape_test)
        # metric_update_track["loss"].append(current_loss)
        # metric_update_track["mse"].append(current_mse)
        # metric_update_track["reg1"].append(current_reg1)
        # metric_update_track["reg2"].append(current_reg2)



#         # Training loop
# num_epochs = 500
# for epoch in range(num_epochs):
#     _, current_error, cuurent_mse, current_m1, current_m2, current_c = sess.run([train_op, error, mse_error, m1, m2, c])
#     if (epoch + 1) % 25 == 0:
#         print(f"Epoch {epoch + 1}/{num_epochs}, Error: {current_error:.4f}, MSE: {cuurent_mse:.4f}, m1: {current_m1}, m2: {current_m2}, c: {current_c}")

# # Print the final results for 'm' and 'c'
# final_m1, final_m2, final_c = sess.run([m1, m2, c])
# print(f"Final 'm1' value: {final_m1}")

# print(f"Final 'm2' value: {final_m2}")
# print(f"Final 'c' value: {final_c}")

Epoch 250/20000, Loss: 543049.9278, WAPE: 0.2030, WAPE_TEST: 0.3192, WAPE_VOL: 0.2052, WAPE_VOL_TEST: 0.2763, reg1: 1398.3324, reg2: 36.3997
Epoch 500/20000, Loss: 534467.1720, WAPE: 0.2019, WAPE_TEST: 0.3048, WAPE_VOL: 0.2046, WAPE_VOL_TEST: 0.2712, reg1: 1431.1765, reg2: 35.4539
Epoch 750/20000, Loss: 531670.2802, WAPE: 0.2018, WAPE_TEST: 0.3091, WAPE_VOL: 0.2042, WAPE_VOL_TEST: 0.2714, reg1: 1386.8642, reg2: 35.3025
Epoch 1000/20000, Loss: 530612.9994, WAPE: 0.2011, WAPE_TEST: 0.3132, WAPE_VOL: 0.2041, WAPE_VOL_TEST: 0.2742, reg1: 1339.7920, reg2: 35.2373
Epoch 1250/20000, Loss: 530132.5024, WAPE: 0.2011, WAPE_TEST: 0.3157, WAPE_VOL: 0.2043, WAPE_VOL_TEST: 0.2767, reg1: 1288.2510, reg2: 35.2193
Epoch 1500/20000, Loss: 536561.1547, WAPE: 0.1981, WAPE_TEST: 0.3216, WAPE_VOL: 0.2080, WAPE_VOL_TEST: 0.2807, reg1: 1258.3257, reg2: 34.9466
Epoch 1750/20000, Loss: 530135.2491, WAPE: 0.2039, WAPE_TEST: 0.3154, WAPE_VOL: 0.2040, WAPE_VOL_TEST: 0.2777, reg1: 1246.2881, reg2: 35.3746
Epoch 200

In [163]:
nr_submit, vol_submit = sess.run([y_pred, y_vol_pred], feed_dict3)
nr_submit = nr_submit * scaler
vol_submit = vol_submit * vol_scaler

nr_data_temp = (
    sales_data
    .reset_index()
    .groupby(["date", "sku"])
    .net_revenue.sum()
    .sort_index()
    .unstack(1)
)
nr_submit = pd.DataFrame(nr_submit, index=nr_data_temp.index[-2:], columns=nr_data_temp.columns)
vol_submit = pd.DataFrame(vol_submit, index=nr_data_temp.index[-2:], columns=nr_data_temp.columns)

submit_temp = sales_data[sales_data.gto.isna()].reset_index().set_index(["date", "sku", "brand", "pack", "size"]).sort_index()
submit_temp.loc[:, "net_revenue"] = submit_temp.net_revenue.fillna(nr_submit.stack()).apply(lambda x: x if x>0 else -x/2)
submit_temp.loc[:, "volume"] = submit_temp.volume.fillna(vol_submit.stack()).apply(lambda x: x if x>0 else -x/2)
submit_temp = submit_temp.reset_index()

cols_req = [ "Year", "Month", "SKU", "Brand", "Pack", "Size", "Volume_Estimate", "Net_Revenue_Estimate", "Optimal_Promotional_Discount", "Optimal_Other_Discounts", "Optimal_Volume", "Optimal_Net_Revenue"]


submit_temp.loc[:, "Year"] = submit_temp.date.dt.year
submit_temp.loc[:, "Month"] = submit_temp.date.dt.month
submit_temp.loc[:, "SKU"] = submit_temp.sku
submit_temp.loc[:, "Brand"] = submit_temp.brand
submit_temp.loc[:, "Pack"] = submit_temp.pack
submit_temp.loc[:, "Size"] = submit_temp.size
submit_temp.loc[:, "Volume_Estimate"] = submit_temp.volume
submit_temp.loc[:, "Net_Revenue_Estimate"] = submit_temp.net_revenue
submit_temp.loc[:, "Optimal_Promotional_Discount"] = submit_temp.promotional_discount
submit_temp.loc[:, "Optimal_Other_Discounts"] = submit_temp.other_discounts
submit_temp.loc[:, "Optimal_Volume"] = submit_temp.volume
submit_temp.loc[:, "Optimal_Net_Revenue"] = submit_temp.net_revenue

submit_temp = submit_temp[cols_req]
submit_temp.to_csv("/home/akshay-development-server/promo-optimization_team-simpsons-paradox/data/team_simpsons_paradox_submission_1.csv", index=False)